In [1]:
# Speech emotion recognition on MSP-IMPROV dataset
# feature: egemaps
# Algorithm: as described in Bagus TA et al., "RNN-based dimenisional SER",
#                            ASJ Autumn 2019.
# 2019-12-24: Modified from ser_improv_egamaps, change input to HFS
#             Filter only natural interaction and improvisation data
# 2019-12-25: Updaate to adapt train/test data-label, change it accordingly
#             to be run in csl directory (although data in improv_loso/)
# 2020-05-26: update for SC paper

import numpy as np
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, \
                         Dropout, Flatten, LSTM, BatchNormalization, \
                         RNN, SimpleRNN, Activation
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
import random as rn
import tensorflow as tf
# from attention_helper import AttentionDecoder
#from read_csv import load_features
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence
import codecs
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# loading file and label, choose the feature type first!
feat = np.load('../improv_loso/data/feat_hfs_mspin_train.npy')
feat_train = feat.reshape(feat.shape[0], 1, feat.shape[1])

feat_test = np.load('../improv_loso/data/feat_hfs_mspin_test.npy')
feat_test = feat_test.reshape(feat_test.shape[0], 1, feat_test.shape[1])

# load train and test ata
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_file = pd.read_csv(list_path_train, index_col=None)

vad_list = [list_file['v'], list_file['a'], list_file['d']]
vad_train = np.array(vad_list).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

vad_test = [list_test ['v'], list_test['a'], list_test['d']]
vad_test = np.array(vad_test).T

feat = np.concatenate([feat_train, feat_test], axis=0)
vad = np.concatenate([vad_train, vad_test])

# uncomment the following to remove silence feature (HFS1)
feat = feat[:,:, :-1]

# load test data
print('Feature shape: ', feat.shape)
print('Label shape: ', vad.shape)


Feature shape:  (7166, 1, 46)
Label shape:  (7166, 3)


In [4]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
nb_words = 3438
MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 3427 unique tokens
../../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 167


In [5]:
## standardization
scaled_vad = True

if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [6]:
# split train/test
split = 5744

In [7]:
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
# model2 = model()

In [8]:
# main function for for LOSO
def main(time):
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict([feat[4596:],x_text[4596:]], batch_size=32)
    np.save('5744npy/st_5744-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2570).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5744npy/st_5744.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 88s 19ms/step - loss: 1.7741 - v_loss: 0.6371 - a_loss: 0.5489 - d_loss: 0.5870 - v_ccc: 0.3628 - a_ccc: 0.4508 - d_ccc: 0.4122 - val_loss: 1.6683 - val_v_loss: 0.6744 - val_a_loss: 0.4674 - val_d_loss: 0.5273 - val_v_ccc: 0.3255 - val_a_ccc: 0.5329 - val_d_ccc: 0.4732
Epoch 2/50
4595/4595 [==============================] - 81s 18ms/step - loss: 1.3207 - v_loss: 0.4809 - a_loss: 0.3736 - d_loss: 0.4651 - v_ccc: 0.5185 - a_ccc: 0.6261 - d_ccc: 0.5347 - val_loss: 1.6092 - val_v_loss: 0.6724 - val_a_loss: 0.4344 - val_d_loss: 0.5023 - val_v_ccc: 0.3272 - val_a_ccc: 0.5656 - val_d_ccc: 0.4980
Epoch 3/50
4595/4595 [==============================] - 83s 18ms/step - loss: 1.1872 - v_loss: 0.4103 - a_loss: 0.3440 - d_loss: 0.4323 - v_ccc: 0.5898 - a_ccc: 0.6557 - d_ccc: 0.5673 - val_loss: 1.5404 - val_v_loss: 0

Epoch 9/50
4595/4595 [==============================] - 86s 19ms/step - loss: 0.6863 - v_loss: 0.2073 - a_loss: 0.2162 - d_loss: 0.2624 - v_ccc: 0.7926 - a_ccc: 0.7838 - d_ccc: 0.7373 - val_loss: 1.7166 - val_v_loss: 0.6570 - val_a_loss: 0.4932 - val_d_loss: 0.5662 - val_v_ccc: 0.3425 - val_a_ccc: 0.5066 - val_d_ccc: 0.4343
Epoch 10/50
4595/4595 [==============================] - 85s 19ms/step - loss: 0.6305 - v_loss: 0.1856 - a_loss: 0.2078 - d_loss: 0.2363 - v_ccc: 0.8141 - a_ccc: 0.7920 - d_ccc: 0.7635 - val_loss: 1.8460 - val_v_loss: 0.6816 - val_a_loss: 0.4834 - val_d_loss: 0.6808 - val_v_ccc: 0.3178 - val_a_ccc: 0.5163 - val_d_ccc: 0.3199
Epoch 11/50
4595/4595 [==============================] - 85s 18ms/step - loss: 0.5810 - v_loss: 0.1743 - a_loss: 0.1898 - d_loss: 0.2172 - v_ccc: 0.8259 - a_ccc: 0.8102 - d_ccc: 0.7829 - val_loss: 1.7585 - val_v_loss: 0.6539 - val_a_loss: 0.4878 - val_d_loss: 0.6164 - val_v_ccc: 0.3456 - val_a_ccc: 0.5119 - val_d_ccc: 0.3841
Epoch 12/50
4595/459

4595/4595 [==============================] - 88s 19ms/step - loss: 0.8033 - v_loss: 0.2431 - a_loss: 0.2534 - d_loss: 0.3061 - v_ccc: 0.7569 - a_ccc: 0.7461 - d_ccc: 0.6936 - val_loss: 1.6710 - val_v_loss: 0.6616 - val_a_loss: 0.4497 - val_d_loss: 0.5590 - val_v_ccc: 0.3376 - val_a_ccc: 0.5501 - val_d_ccc: 0.4413
Epoch 8/50
4595/4595 [==============================] - 88s 19ms/step - loss: 0.7412 - v_loss: 0.2225 - a_loss: 0.2412 - d_loss: 0.2773 - v_ccc: 0.7781 - a_ccc: 0.7584 - d_ccc: 0.7223 - val_loss: 1.6903 - val_v_loss: 0.6601 - val_a_loss: 0.4525 - val_d_loss: 0.5771 - val_v_ccc: 0.3393 - val_a_ccc: 0.5471 - val_d_ccc: 0.4233
Epoch 9/50
4595/4595 [==============================] - 88s 19ms/step - loss: 0.6733 - v_loss: 0.2015 - a_loss: 0.2209 - d_loss: 0.2516 - v_ccc: 0.7989 - a_ccc: 0.7793 - d_ccc: 0.7485 - val_loss: 1.6987 - val_v_loss: 0.6633 - val_a_loss: 0.4621 - val_d_loss: 0.5727 - val_v_ccc: 0.3360 - val_a_ccc: 0.5376 - val_d_ccc: 0.4276
Epoch 10/50
4595/4595 [==========

4595/4595 [==============================] - 89s 19ms/step - loss: 0.9206 - v_loss: 0.2875 - a_loss: 0.2855 - d_loss: 0.3483 - v_ccc: 0.7127 - a_ccc: 0.7149 - d_ccc: 0.6518 - val_loss: 1.6220 - val_v_loss: 0.6552 - val_a_loss: 0.4495 - val_d_loss: 0.5172 - val_v_ccc: 0.3441 - val_a_ccc: 0.5505 - val_d_ccc: 0.4834
Epoch 7/50
4595/4595 [==============================] - 87s 19ms/step - loss: 0.8285 - v_loss: 0.2570 - a_loss: 0.2564 - d_loss: 0.3157 - v_ccc: 0.7434 - a_ccc: 0.7434 - d_ccc: 0.6847 - val_loss: 1.6968 - val_v_loss: 0.6867 - val_a_loss: 0.4635 - val_d_loss: 0.5465 - val_v_ccc: 0.3127 - val_a_ccc: 0.5365 - val_d_ccc: 0.4540
Epoch 8/50
4595/4595 [==============================] - 86s 19ms/step - loss: 0.7582 - v_loss: 0.2295 - a_loss: 0.2362 - d_loss: 0.2931 - v_ccc: 0.7708 - a_ccc: 0.7640 - d_ccc: 0.7069 - val_loss: 1.8220 - val_v_loss: 0.7060 - val_a_loss: 0.4906 - val_d_loss: 0.6255 - val_v_ccc: 0.2934 - val_a_ccc: 0.5094 - val_d_ccc: 0.3752
Epoch 9/50
4595/4595 [===========

4595/4595 [==============================] - 88s 19ms/step - loss: 1.0766 - v_loss: 0.3545 - a_loss: 0.3171 - d_loss: 0.4043 - v_ccc: 0.6454 - a_ccc: 0.6824 - d_ccc: 0.5956 - val_loss: 1.5660 - val_v_loss: 0.6233 - val_a_loss: 0.4342 - val_d_loss: 0.5084 - val_v_ccc: 0.3761 - val_a_ccc: 0.5658 - val_d_ccc: 0.4922
Epoch 5/50
 192/4595 [>.............................] - ETA: 1:18 - loss: 0.9594 - v_loss: 0.2774 - a_loss: 0.2899 - d_loss: 0.3921 - v_ccc: 0.7226 - a_ccc: 0.7101 - d_ccc: 0.6079

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4595/4595 [==============================] - 100s 22ms/step - loss: 1.0890 - v_loss: 0.3564 - a_loss: 0.3254 - d_loss: 0.4066 - v_ccc: 0.6434 - a_ccc: 0.6743 - d_ccc: 0.5934 - val_loss: 1.5448 - val_v_loss: 0.6196 - val_a_loss: 0.4220 - val_d_loss: 0.5029 - val_v_ccc: 0.3797 - val_a_ccc: 0.5779 - val_d_ccc: 0.4976
Epoch 5/50
4595/4595 [==============================] - 101s 22ms/step - loss: 1.0056 - v_loss: 0.3115 - a_loss: 0.3059 - d_loss: 0.3893 - v_ccc: 0.6884 - a_ccc: 0.6943 - d_ccc: 0.6116 - val_loss: 1.6976 - val_v_loss: 0.6523 - val_a_loss: 0.4829 - val_d_loss: 0.5622 - val_v_ccc: 0.3472 - val_a_ccc: 0.5171 - val_d_ccc: 0.4382
Epoch 6/50
4595/4595 [==============================] - 101s 22ms/step - loss: 0.9191 - v_loss: 0.2802 - a_loss: 0.2837 - d_loss: 0.3550 - v_ccc: 0.7195 - a_ccc: 0.7162 - d_ccc: 0.6452 - val_loss: 1.6371 - val_v_loss: 0.6500 - val_a_loss: 0.4483 - val_d_loss: 0.5388 - val_v_ccc: 0.3493 - val_a_ccc: 0.5517 - val_d_ccc: 0.4619
Epoch 7/50
4595/4595 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4595/4595 [==============================] - 114s 25ms/step - loss: 1.7852 - v_loss: 0.6782 - a_loss: 0.5232 - d_loss: 0.5839 - v_ccc: 0.3222 - a_ccc: 0.4767 - d_ccc: 0.4159 - val_loss: 1.6974 - val_v_loss: 0.6421 - val_a_loss: 0.4890 - val_d_loss: 0.5664 - val_v_ccc: 0.3575 - val_a_ccc: 0.5111 - val_d_ccc: 0.4340
Epoch 2/50
4595/4595 [==============================] - 100s 22ms/step - loss: 1.3299 - v_loss: 0.4873 - a_loss: 0.3794 - d_loss: 0.4656 - v_ccc: 0.5143 - a_ccc: 0.6211 - d_ccc: 0.5347 - val_loss: 1.5823 - val_v_loss: 0.6196 - val_a_loss: 0.4407 - val_d_loss: 0.5218 - val_v_ccc: 0.3799 - val_a_ccc: 0.5594 - val_d_ccc: 0.4785
Epoch 3/50
4595/4595 [==============================] - 100s 22ms/step - loss: 1.1997 - v_loss: 0.4093 - a_loss: 0.3489 - d_loss: 0.4421 - v_ccc: 0.5911 - a_ccc: 0.6509 - d_ccc: 0.5583 - val_loss: 1.5309 - val_v_loss: 0.6051 - val_a_loss: 0.4194 - val_d_loss: 0.5063 - val_v_ccc: 0.3943 - val_a_ccc: 0.5807 - val_d_ccc: 0.4941
Epoch 4/50
4595/4595 [========

Epoch 13/50
1422/1422 [==============================] - 9s 6ms/step
[0.3242606520652771, 0.5454472303390503, 0.3973333239555359]
18
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 112s 24ms/step - loss: 1.7977 - v_loss: 0.6695 - a_loss: 0.5263 - d_loss: 0.6005 - v_ccc: 0.3304 - a_ccc: 0.4726 - d_ccc: 0.3992 - val_loss: 1.9009 - val_v_loss: 0.6901 - val_a_loss: 0.5875 - val_d_loss: 0.6235 - val_v_ccc: 0.3098 - val_a_ccc: 0.4125 - val_d_ccc: 0.3769
Epoch 2/50
4595/4595 [==============================] - 98s 21ms/step - loss: 1.3415 - v_loss: 0.4963 - a_loss: 0.3805 - d_loss: 0.4642 - v_ccc: 0.5033 - a_ccc: 0.6196 - d_ccc: 0.5356 - val_loss: 1.6111 - val_v_loss: 0.6623 - val_a_loss: 0.4401 - val_d_loss: 0.5088 - val_v_ccc: 0.3372 - val_a_ccc: 0.5600 - val_d_ccc: 0.4917
Epoch 3/50
4595/4595 [==============================] - 98s 21ms/step - loss: 1.1880 - v_loss: 0.4059 - a_loss: 0.3433 - d_loss: 0.4382 - v_ccc: 0.5937 - a_ccc: 0.656

In [9]:
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
# hist = model2.fit([feat[:split],x_text[:split]], vad[:split].T.tolist(), batch_size=32,
#                   validation_split=0.2, epochs=50, verbose=1, shuffle=True,
#                   callbacks=[earlystop])

# # outs =loss, mse_v, mse_a, mse_d, ccc_v, ccc_a, ccc_d
# # test data start from 5744/split
# metrik2 = model2.evaluate([feat[split:],x_text[split:]], vad[split:].T.tolist())#, a[8000:], d[8000:]])
# print(metrik2[-3:])

# # # make prediction
# # feat_predict = feat[4596:]
# # vad_predict = vad[4596:]

# predict2 = model2.predict([feat[4596:],x_text[4596:]], batch_size=32)
# np.save('st_glove_npy_loso',  np.array(predict2).reshape(3, 2570).T)